In [1]:
import os
os.environ['KERAS_BACKEND'] = "torch"
import keras
import keras.backend as K
from keras import layers, Model
from keras.optimizers import Adam


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from modules.dataset import DatasetFactory
from modules.encoding import NLFEncoder

2024-03-23 12:04:33.021138: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 12:04:33.206582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 12:04:33.206616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 12:04:33.223196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 12:04:33.259076: I tensorflow/core/platform/cpu_feature_guar

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataset_dir = "../data/SAbDab"

df_match = pd.read_csv(os.path.join(dataset_dir, "data.csv"), sep=";")
df_seq = pd.read_csv(os.path.join(dataset_dir, "sequences.csv"), sep=";")

In [4]:
df_match.head()

,ab,ag,interaction
0,5kel|ab,5kel|ag,1
1,5kel|ab,6cwt|ag,0
2,5kel|ab,4fp8|ag,0
3,5kel|ab,4yjz|ag,0
4,5kel|ab,6j15|ag,0


### Filter 

Remove keys which does not have both an antigen and an anticorps.

In [5]:
df_filtered = df_match[df_match["ab"].isin(df_seq["seq_id"]) & df_match["ag"].isin(df_seq["seq_id"])]
df_filtered.shape

(345323, 3)

In [6]:
df_match.shape

(421821, 3)

## Encoding

### NLF

We want to encode the sequence first, and retrieve the encoded for each sequence in the join later.

In [7]:
encoder = NLFEncoder()

In [8]:
seq = df_seq["sequence"].values
encoded_seq = encoder.encode(seq)

In [9]:
seq_id_to_nlf = {seq_id: nlf for seq_id, nlf in zip(df_seq["seq_id"], encoded_seq)}

In [10]:

x_ag = df_filtered["ag"].map(lambda x: seq_id_to_nlf[x])
x_ab = df_filtered["ab"].map(lambda x: seq_id_to_nlf[x])

In [11]:
input_dimensions = x_ag[0].shape

In [12]:
# Split both datasets
X = pd.DataFrame({"ag": x_ag, "ab": x_ab})
y = df_filtered["interaction"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
x_train_ag = np.stack(x_train["ag"].values)
x_train_ab = np.stack(x_train["ab"].values)

In [14]:
x_train_ag.shape, x_train_ab.shape

((276258, 1500, 18), (276258, 1500, 18))

## Siamese network

In [15]:
seq_input1 = layers.Input(shape=input_dimensions, name='seq_ag')
seq_input2 = layers.Input(shape=input_dimensions, name='seq_ab')

In [16]:
# Convolutional modules
filters = 96

conv01 = layers.Conv1D(filters, 11, padding='same', activation="relu")
mp1 = layers.MaxPooling1D(3)
conv02 = layers.Conv1D(filters*2, 7, padding='same', activation="relu")
mp2 = layers.MaxPooling1D(3)
conv03 = layers.Conv1D(filters*4, 3, padding='same', activation="relu")
mp3 = layers.MaxPooling1D(3)
conv04 = layers.Conv1D(filters*2, 3, padding='same', activation="relu")
mp4 = layers.MaxPooling1D(3)

gru = layers.Bidirectional(layers.GRU(filters, return_sequences=False))

def siamese_propagation(x):
    x = conv01(x)
    x = mp1(x)

    x = conv02(x)
    x = mp2(x)

    x = conv03(x)
    x = mp3(x)

    x = conv04(x)
    x = mp4(x)

    x_gru = gru(x)
    return x_gru

2024-03-23 12:04:56.173341: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [17]:
def forward(left, right):
    left = siamese_propagation(left)
    right = siamese_propagation(right)

    merge = layers.multiply([left, right])

    # merge = layers.Dense(filters*2, activation='relu')(merge)
    merge = layers.Dropout(0.2)(merge)
    return layers.Dense(1, activation='sigmoid')(merge)

In [18]:
def f1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    pred_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pred_pos + K.epsilon())
    recall = tp / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def mcc(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return numerator / (denominator + K.epsilon())

def accuracy(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    return (tp + tn) / (tp + tn + fp + fn)

In [19]:
def binary_crossentropy(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    loss = - K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
    return loss

In [20]:
model = Model(inputs=[seq_input1, seq_input2],
            outputs=[forward(seq_input1, seq_input2)])

adam = Adam(learning_rate=1e-4, amsgrad=True, epsilon=1e-6)

checkpoint_callback = keras.callbacks.ModelCheckpoint('run/siamese/model.h5', monitor='val_mcc', mode='max')
earlystop_callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model.compile(optimizer=adam, loss=binary_crossentropy, metrics=[accuracy, f1, mcc])

model.fit([x_train_ab, x_train_ag], y_train, epochs=50, callbacks=[checkpoint_callback, earlystop_callback],
          batch_size=64, verbose=1, validation_split=0.15)
